In [1]:
from pororo import Pororo
import json

In [2]:
se = Pororo(task="sentence_embedding", lang="ko")

In [3]:
train_file = '../../data/step1_train.json' 
validation_file = '../../data/step1_valid.json'
model_name_or_path = '../../data/models/electra'

In [4]:
with open(train_file, 'r', encoding = 'UTF-8') as fp:
    train_data = json.load(fp)
with open(validation_file, 'r', encoding = 'UTF-8') as fp:
    test_data = json.load(fp)

In [5]:
label_lst= []
cat_lst = []
cont_lst = []
for i in range(len(train_data['input_data'])):
    label_lst.append(train_data['input_data'][i]['label'])
    cat_lst.append(train_data['input_data'][i]['category'])
    cont_lst.append(train_data['input_data'][i]['content']) 
    
for i in range(len(test_data['input_data'])):
    label_lst.append(test_data['input_data'][i]['label'])
    cat_lst.append(test_data['input_data'][i]['category'])
    cont_lst.append(test_data['input_data'][i]['content']) 

In [6]:
cont_proc_lst = []
for i in cont_lst:
    cont_proc_lst.append(se(i))

In [7]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
label = le.fit_transform(cat_lst)

In [8]:
keys = le.classes_
values = le.transform(le.classes_)
dictionary = dict(zip(keys, values))

In [9]:
dictionary

{'IT/과학': 0,
 '경제': 1,
 '대북정책 폄훼': 2,
 '대중의 관심 유도': 3,
 '로열패밀리 신변이상': 4,
 '사설': 5,
 '사회': 6,
 '생활/문화': 7,
 '정치': 8,
 '최고지도자 신변이상': 9,
 '최고지도자 측근(당·정·군 관료) 숙청': 10}

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(cont_proc_lst, label, test_size = 0.2,stratify = label,shuffle=True, random_state=2 )

In [14]:
# LDA gives the best result
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

clf = LinearDiscriminantAnalysis()
clf.fit(X_train, y_train)
ypred=clf.predict(X_test)

In [15]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
result = confusion_matrix(y_test, ypred)
print('Confusion Matrix: ')
print(result)
result1 = classification_report(y_test, ypred)
print('Classification Report: ')
print (result1)
result2 = accuracy_score(y_test,ypred)
print(f'Accuracy: {result2}')

Confusion Matrix: 
[[  1   0   0   0   0   0   1   0   1   0   0]
 [  0  17   0   0   0   0   4   2   5   0   1]
 [  0   0   6   0   0   0   4   0   2   0   1]
 [  0   0   0  23   0   0   3   0   5   1   0]
 [  0   0   0   1  37   0   1   0   0   3   0]
 [  0   2   0   0   0   0   1   0   0   0   0]
 [  0   9   2   4   0   0 127   6  31   2   1]
 [  0   0   0   0   0   0   4  12   2   0   1]
 [  0   7   0   2   0   0  26   2 106   1   2]
 [  0   0   0   2   0   0   6   0   3  75   1]
 [  0   0   1   1   0   0   1   1   6   0  62]]
Classification Report: 
              precision    recall  f1-score   support

           0       1.00      0.33      0.50         3
           1       0.49      0.59      0.53        29
           2       0.67      0.46      0.55        13
           3       0.70      0.72      0.71        32
           4       1.00      0.88      0.94        42
           5       0.00      0.00      0.00         3
           6       0.71      0.70      0.71       182
      

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
# spot check machine learning algorithms on the glass identification dataset
from numpy import mean
from numpy import std
from pandas import read_csv
from matplotlib import pyplot
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import KFold
def get_models():
    models, names = list(), list()
    # SVM
    models.append(LinearSVC())
    names.append('SVM')
    # KNN
    models.append(KNeighborsClassifier())
    names.append('KNN')

    # RF
    models.append(RandomForestClassifier(n_estimators=1000))
    names.append('RF')
    # ET
    models.append(ExtraTreesClassifier(n_estimators=1000))
    names.append('ET')

    return models, names
def evaluate_model(X, y, model):
    # define evaluation procedure
    #cv = KFold(n_splits=5, shuffle=True)
    # evaluate model
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=5, n_jobs = 2)
    return scores

# define models
models, names = get_models()
results = list()
# evaluate each model
for i in range(len(models)):
    # evaluate the model and store results
    scores = evaluate_model(cont_proc_lst, label, models[i])
    results.append(scores)
    # summarize performance
    print('>%s %.3f (%.3f)' % (names[i], mean(scores), std(scores)))

>SVM 0.733 (0.009)
>KNN 0.714 (0.014)
>RF 0.669 (0.029)


/opt/conda/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


>ET 0.672 (0.028)
